In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

import os
import argparse
import numpy as np



In [ ]:
# Prepare the data

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

0it [00:00, ?it/s]

170500096it [00:06, 27260666.76it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


def test():
    net = VGG('VGG11')
    x = torch.randn(2,3,32,32)
    y = net(x)
    print(y.size())

# test()


In [ ]:

print('==> Building model..')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = VGG('VGG11')

net = net.to(device)

if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)


# Training

def train(epoch):
    print('\n When Epoch is : %d' % epoch,' Total epoch =10')
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'% (train_loss/(batch_idx+1), 100.*correct/total, correct, total),"Rate of progress ",(total/50000)*100,"%")
        
        

def test(epoch):
    #global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            print('Finished train!')
            print('Using testset to test the model.')
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total),"Rate of progress  ",(total/10000)*100,"%")
    list_acc = []            
    acc = 100.*correct/total
    list_acc.append(acc)
    best_acc = max(list_acc)
    print('The best acc is in total is ',best_acc,"when epoch = ",epoch,"%")
            

start_epoch = 0
            
if __name__ == '__main__':
    for epoch in range(start_epoch, start_epoch+10):
        train(epoch)
        test(epoch)

==> Building model..

 When Epoch is : 0  Total epoch =10
Loss: 2.465 | Acc: 4.688% (6/128) Rate of progress  0.256 %
Loss: 2.905 | Acc: 9.766% (25/256) Rate of progress  0.512 %
Loss: 4.662 | Acc: 11.979% (46/384) Rate of progress  0.768 %
Loss: 6.132 | Acc: 11.133% (57/512) Rate of progress  1.024 %
Loss: 6.619 | Acc: 10.781% (69/640) Rate of progress  1.28 %
Loss: 7.374 | Acc: 10.547% (81/768) Rate of progress  1.536 %
Loss: 7.309 | Acc: 10.714% (96/896) Rate of progress  1.7919999999999998 %
Loss: 7.319 | Acc: 10.840% (111/1024) Rate of progress  2.048 %
Loss: 7.148 | Acc: 10.417% (120/1152) Rate of progress  2.3040000000000003 %
Loss: 6.880 | Acc: 10.000% (128/1280) Rate of progress  2.56 %
Loss: 6.813 | Acc: 10.014% (141/1408) Rate of progress  2.8160000000000003 %
Loss: 6.610 | Acc: 10.286% (158/1536) Rate of progress  3.072 %
Loss: 6.397 | Acc: 10.517% (175/1664) Rate of progress  3.328 %
Loss: 6.225 | Acc: 10.882% (195/1792) Rate of progress  3.5839999999999996 %
Loss: 6.128 |